<a href="https://colab.research.google.com/github/Nathan-Roll1/ProsodPy/blob/main/Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!git clone https://github.com/Nathan-Roll1/ProsodPy

from ProsodPy.ProsodPy import *
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import librosa   
import pickle

plt.rcParams["figure.figsize"] = (20,3)

Cloning into 'ProsodPy'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 46 (delta 9), reused 18 (delta 4), pack-reused 0
Unpacking objects: 100% (46/46), done.


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [ ]:
init_path = 'drive/Shareddrives/Ling Thesis/resampled/'
fs = os.listdir(init_path)
fs = [init_path + x for x in fs]

In [ ]:
for n, f in enumerate(fs):
  print(n,f)
  text_path = 'drive/Shareddrives/Rezonator Data/SBCSAE/'
  tt = f.split('/')[-1].split('_')[0]
  txt = text_path + tt +'.cha'
  boundaries = get_boundaries(txt)
  boundaries = boundaries*8000 # Adjust for 8k sample rate

  with open(f, 'rb') as handle:
    audio_array = pickle.load(handle)
    
  pre_mfcc, post_mfcc, boundary_mfcc, labels = MFCC_preprocess(audio_array, boundaries)

  size = sum(labels==1)
  indicies = list(np.random.choice(np.where(labels == 0)[0], int(size))) #Generate (n) Random False Labels
  indicies += list(np.where(labels == 1)[0]) #Create Balanced Mask

  labels = labels[indicies] #Apply Mask to Labels
  pre_mfcc = pre_mfcc[indicies]
  post_mfcc = post_mfcc[indicies]
  boundary_mfcc = boundary_mfcc[indicies]

  for t in ['pre_mfcc','post_mfcc','boundary_mfcc']:
    with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/{t}/{tt}_cont.pickle', 'wb') as handle:
      pickle.dump(globals()[t], handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/labels/{tt}_cont.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)